Automating data gethering from the internet to the database

GANS PROJECT For the Gans Project we need to work on getting information from avrious resources into our DB tables for further analysis Starting with Getting information about flights for major cities in germany. Using the AeroDataBoxAPI we get the flights information.

In [ ]:
#Flights API
#Getting the arrivals for major cities in Germany form the AeroDataBoxAPI

import requests
from datetime import datetime
from datetime import timedelta
import pandas as pd
from IPython.display import JSON

#API key from the AeroDataBoxAPI
API_key = "API_KEY"

# Airports icoa
list_airport_icoa = ["EDDB","EDDF","EDDM","EDDK","EDDS"]


#company wants to know which flights will arrive the next day -Date Format.
#Add 1 day.
tomorrows_date = (datetime.now() + timedelta(days=1))
tomorrows_date_from=tomorrows_date.strftime("%Y-%m-%dT%H:%M")
#add 11 hours
future_date_and_time = tomorrows_date  + timedelta(hours=11)
tomorrows_date_to = future_date_and_time.strftime("%Y-%m-%dT%H:%M")
def get_flight_info(flight_json,i):
    # terminal
    try: terminal = flight_json['arrival']['terminal']
    except: terminal = None
    # aircraft
    try: aircraft = flight_json['aircraft']['model']
    except: aircraft = None

    return {
        'dep_airport':flight_json['departure']['airport']['name'],
        'sched_arr_loc_time':flight_json['arrival']['scheduledTimeLocal'],
        'terminal':terminal,
        'status':flight_json['status'],
        'aircraft':aircraft,
        'icao_code':i
    }
for i in list_airport_icoa:
    print(i)
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{i}/{tomorrows_date_from}/{tomorrows_date_to}"
    querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true",
                   "withCargo":"true","withPrivate":"false","withLocation":"false"}

    headers = {
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
        "X-RapidAPI-Key": API_key
    }
    response = requests.request("GET", url, headers=headers, params=querystring)
    arrivals = response.json()['arrivals']
    appended_data=pd.concat([appended_data, pd.DataFrame([get_flight_info(flight,i) for flight in arrivals])], ignore_index=True)